In [611]:
import pandas as pd
from requests_html import HTMLSession
from tqdm.notebook import tqdm
import re

In [2]:
dr_num_list = ["pertama"] + [i for i in range(2, 12)]

In [19]:
# dr_num = 11
table_combined = pd.DataFrame(
    columns=["kod_kawasan", "kawasan", "nama", "parti", "negeri", "vol"]
)
for dr_num in tqdm(dr_num_list):
    if dr_num == "pertama":
        url = (
            "https://ms.wikipedia.org/wiki/Ahli_Dewan_Rakyat,_Parlimen_Malaysia_pertama"
        )
    else:
        url = f"https://ms.wikipedia.org/wiki/Ahli_Dewan_Rakyat,_Parlimen_Malaysia_ke-{dr_num}"
    dr_df = pd.read_html(url)
    col_check = [
        ["No.", "Kawasan Parlimen", "Wakil Rakyat (Ahli Parlimen)", "Parti (Gabungan)"],
        ["No.", "Kawasan Parlimen", "Wakil Rakyat", "Parti"],
        ["No.", "Kawasan Parlimen", "Nama", "Parti"],
        ["No.", "Kawasan Parlimen", "Ahli", "Parti"],
        ["No.", "Kawasan persekutuan", "Ahli", "Parti"],
        ["No.", "Kawasan", "Ahli", "Parti"],
        ["No.", "Kawasan", "Ahli", "Parti", "Catatan"],
        ["No.", "Parlimen", "Ahli", "Parti"],
        ["No.", "Federal Constituency", "Member", "Party"],
        ["No.", "Parliament Constituency", "Member", "Party (Coalition)"],
    ]
    all_Table = []
    for t in dr_df:
        # print(t.columns.to_list())
        if t.columns.to_list() == ["No.", "Kawasan", "Ahli", "Unnamed: 3"]:
            t.columns = ["No.", "Kawasan", "Ahli", "Parti"]
        if t.columns.to_list() in col_check:
            all_Table.append(t)
    with HTMLSession() as session:
        re = session.get(url)
    state_list = [
        i.text.split("\n")[0]
        for i in re.html.xpath("//*[@class = 'mw-heading mw-heading3']")
    ]
    if dr_num in ["pertama", 2]:
        state_list = state_list[:-3]
    # elif dr_num == 2:

    if len(state_list) == len(all_Table):
        for s, st in zip(state_list, all_Table):
            st["negeri"] = s
            if dr_num == "pertama":
                st["vol"] = 1
            else:
                st["vol"] = dr_num
            if "Catatan" in st.columns:
                st = st.drop(columns=["Catatan"])
            st.columns = ["kod_kawasan", "kawasan", "nama", "parti", "negeri", "vol"]
            table_combined = pd.concat([table_combined, st[1:]])
    else:
        print(url)
        print(state_list)
        print(len(all_Table))

  0%|          | 0/11 [00:00<?, ?it/s]

In [20]:
table_combined = table_combined.reset_index(drop=True)

In [661]:
# table_combined.to_csv("dr1_11_wiki.csv", index=False)

In [698]:
# table_combined = pd.read_csv("dr1_11_wiki.csv")
table_combined = pd.read_csv("dr1_11_wiki.csv")
kawasan_mapping = table_combined[
    ["kod_kawasan", "kawasan", "negeri", "vol"]
].drop_duplicates()
repalce_dict = {
    # r"\(.*\)":"",
    # "Sungei": "Sungai",
    # "Rembau-Tampin": "Rembau",
    "Jelebu-Jempol": "Jelebu",
    "Kramat": "Keramat",
    "Alor Star": "Alor Setar",
    "Kota Bharu": "Kota Baru",
    "Kota Baru Hilir": "Kota Bharu Hilir",
    "Trengganu": "Terengganu",
    "Jitra-Padang Terap": "Jitra",
    "Bungsar": "Bangsar",
    "Patani": "Petani",
    "Malacca": "Melaka",
    "Johore": "Johor",
    "Timor": "Timur",
    # "Tanjong": "Tanjung",
    # "Telok": "Teluk",
    # "P.J. Selatan": "Petaling Jaya Selatan",
}
for k, v in repalce_dict.items():
    kawasan_mapping["kawasan"] = kawasan_mapping["kawasan"].str.replace(k, v)
kawasan_mapping[kawasan_mapping["vol"] == 2].index
replace_vol_dict = {
    1:{
        "Kulim-Bandar Bahru": "Kulim",
        "Rembau-Tampin": "Rembau"
    },
    2: {
        "Kulim": "Kulim Utara",
        "Kulim Utara-Bandar Bahru": "Kulim",
        "Sungei": "Sungai",
        "Johor Bahru": "Johor Bharu",
        "Bangsar": "Bungsar",
        "Petani": "Patani",
        "Tanjung": "Tanjong",
        "Dato' Keramat": "Dato Kramat",
        "Tanjong Malim": "Tanjung Malim",
        "Kota Baru Hulu": "Kota Bharu Hulu"
    },
    3:{
        "Johor Timur":"Johor Timor",
        "Terengganu": "Trengganu",
        "Labuk-Sugut": "Labuk",
        "Sungei Petani": "Sungai Petani",
        "Limbang-Lawas": "Limbang",
        "Seremban Timur": "Seremban Timor",
        "Penang": "Pulau Pinang",
        "Dato' Keramat": "Dato Kramat",
        "Alor Setar": "Alor Star",
        "Labuan-Beaufort": "Labuan",
        "Kota Baru Hulu" :"Kota Bharu Hulu",
        "Trengganu Tengah":"Terengganu Tengah",
        "Lubok Antu": "Luboh Antu",
        "Bau-Lundu": "Bau Lundu",
        "Rembau-Tampin": "Rembau Tampin",
        "Kulim-Bandar Bahru": "Kulim Bandar Bharu",
        "Miri-Subis": "Miri",
        "Padawan": "Pandawan"
    },
    4:{
        "Pelabuhan Kelang": "Pelabohan Klang",
        "Damansara": "Damanasara",
        "Labuk-Sugut": "Labuk Sugut",
        "Kulim-Bandar Bahru": "Kulim Bandar Bahru",
        "Renggam":"Rengam",
        # "Sungei Besi":"Sungai Besi",
        "Sungei":"Sungai",
        "Johor Bahru":"Johor Bharu",
        "Jerlun-Langkawi": "Jerlun",
        "Bau-Lundu": "Bau",
        "Limbang-Lawas": "Limbang",
        # "Sungei Siput": "Sungai Siput",
        "Bruas": "Beruas",
        "Miri-Subis": "Miri",
    },
    5:{
        "Telok": "Teluk",
        "Jerlun-Langkawi":"Jerlun Langkawi",
        "Sungei": "Sungai",
        "Tanjong Malim": "Tanjung Malim",
        "Kulim-Bandar Bahru": "Kulim",
        "Tanjong Karang":"Tanjung Karang",
        "Labuk-Sugut": "Labuk Sugut",
        "Kluang": "Keluang"
    },
    6:{
        "Bruas": "Beruas",
        "Sungei Besi": "Sungai Besi",
        "Sungei Petani": "Sungai Petani",
        "Kulim-Bandar Bahru": "Kulim Bandar Bahru",
        "Labuk-Sugut": "Labuk Sugut",
        "Jerlun-Langkawi": "Jerlun"
    },
    7:{
        "Sungai Siput": "Sungei Siput",
        "Kulim-Bandar Baharu": "Kulim",
        "Labuk-Sugut": "Labuk Sugut",
        "Jerlun-Langkawi":"Jerlun Langkawi",
        "Lubok Antu": "Lubuk Antu",
        "Rasah": "Resah",
        "Tanjong Karang":"Tanjung Karang",
        "Seputeh": "Seputih",
        "Kuala Krai": "Kuala Kerai",
        "Tasek Chenderoh":"Tasik Chenderoh"
    },
    8:{
        "Jerlun-Langkawi":"Jerlun Langkawi",
        "Temerloh": "Temerluh",
        "Bentong": "Bentung",
        "Gopeng": "Gopeh",
        "Kulim-Bandar Baharu": "Kulim",
        "Tasek Gelugor":"Tasik Gelugor",
        "Kota Samarahan": "Kota Semarahan",
        "Bukit Mertajam": "Bukti Mertajam",
        "Kota Baru" :"Kota Bharu",
        "Kota Belud" :"Kota Belut",
        "Johor Bahru": "Johor Baru",
        "Telok Intan": "Teluk Intan",
        "Parit Sulong": "Parit Sulung"
    },
    9:{
        "Kulim-Bandar Baharu": "Kulim Bandar Baharu",
        "Kota Baru" :"Kota Baharu",
        "Johor Bahru": "Johor Baru",
        "Telok Intan": "Teluk Intan",
        "Parit Sulong": "Parit Sulung"
    },
    10:{
        "Kulim-Bandar Baharu": "Kulim",
        "Kota Baru" :"Kota Baharu",
        "Johor Bahru": "Johor Baru",
        "Telok Intan": "Teluk Intan",
        "Parit Sulong": "Parit Sulung",
        "Petaling Jaya Utara": "Petaling Jaya",
        "Kelang":"Klang"
    },
    11:{
        "Kulim-Bandar Baharu": "Kulim Bandar Baharu",
        "Kota Baru" :"Kota Bharu",
        "Telok Intan": "Teluk Intan",
        "Petaling Jaya Selatan": "P.J. Selatan",
        "Tanjong Karang":"Tanjung Karang"
    },
}
for kk,vv in replace_vol_dict.items():
    to_fix_index = kawasan_mapping[kawasan_mapping["vol"] == kk].index
    for kkk, vvv in vv.items():
        kawasan_mapping.loc[to_fix_index, "kawasan"] = kawasan_mapping.loc[to_fix_index, "kawasan"].str.replace(kkk, vvv)

In [699]:
kawasan_mapping[kawasan_mapping["vol"]==7][kawasan_mapping["kod_kawasan"]=="P016"]

/var/folders/7q/vk27jw6d7_95l168mby_n6zc0000gn/T/ipykernel_78014/596844410.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kawasan_mapping[kawasan_mapping["vol"]==7][kawasan_mapping["kod_kawasan"]=="P016"]


,kod_kawasan,kawasan,negeri,vol
872,P016,Kulim,Kedah,7


In [700]:
dr_old = pd.read_csv("DR - DR 1-11.csv")
merged = pd.merge(dr_old.drop(columns=["kod_kawasan","negeri"]).reset_index(), kawasan_mapping.drop_duplicates(), "left", ["vol", "kawasan"])
test_df = merged[merged["kod_kawasan"].isna()][merged["jawatan_parlimen"].isna()]
test_df[test_df["vol"] == 11][~test_df["kawasan"].isin(["Sarawak","Sabah","Singapore"])]

/var/folders/7q/vk27jw6d7_95l168mby_n6zc0000gn/T/ipykernel_78014/2509998756.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = merged[merged["kod_kawasan"].isna()][merged["jawatan_parlimen"].isna()]
/var/folders/7q/vk27jw6d7_95l168mby_n6zc0000gn/T/ipykernel_78014/2509998756.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df[test_df["vol"] == 11][~test_df["kawasan"].isin(["Sarawak","Sabah","Singapore"])]


,index,vol,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti,jawatan_kabinet,jawatan_parlimen,gambar,kod_kawasan,negeri
1643,1643,11,NaN,Senator Tan Sri,Nor Mohamed Bin Yakcop (Dilantik),NaN,male,bumiputera,NaN,NaN,NaN,NaN,NaN,NaN
1657,1657,11,NaN,Senator Dato Sri,Mohd. Effendi Bin Norwawi (Dilantik),NaN,male,bumiputera,NaN,NaN,NaN,NaN,NaN,NaN
1660,1660,11,Kuala Langat (Sehingga 14 Februari 2006),Dato' Sri Dr. Haji,Shafie Bin Haji Mohd. Salleh,NaN,male,bumiputera,NaN,NaN,NaN,NaN,NaN,NaN


In [701]:
merged.iloc[1407:1409]

,index,vol,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti,jawatan_kabinet,jawatan_parlimen,gambar,kod_kawasan,negeri
1407,1407,9,Bayan Baru,Tuan,Wong Kam Hoong,NaN,male,NaN,NaN,NaN,NaN,NaN,P050,Pulau Pinang
1408,1408,9,Serdang,Dato',Yap Pian Hon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P096,Selangor


In [703]:
merged[merged["index"].duplicated()]

,index,vol,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti,jawatan_kabinet,jawatan_parlimen,gambar,kod_kawasan,negeri


In [704]:
final_df = merged[['vol', 'kod_kawasan', 'negeri', 'kawasan', 'gelaran', 'nama',
       'tahun_lahir', 'jantina', 'kumpulan_etnik', 'parti', 'jawatan_kabinet',
       'jawatan_parlimen', 'gambar']]

In [705]:
final_df

,vol,kod_kawasan,negeri,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti,jawatan_kabinet,jawatan_parlimen,gambar
0,1,P007,Kedah,Kuala Kedah,Y.T.M. Tunku,Abdul Rahman Putra AlHaj,NaN,male,bumiputera,NaN,Perdana Menteri,NaN,NaN
1,1,P062,Pahang,Pekan,Y.A.B. Tun,Abdul Razak Bin Dato' Hussain,NaN,male,bumiputera,NaN,Timbalan Perdana Menteri,NaN,NaN
2,1,P102,Johor,Johor Timur,Dato' Dr.,Ismail Bin Dato' Abdul Rahman,NaN,male,bumiputera,NaN,NaN,NaN,NaN
3,1,P046,Perak,Sungei Siput,Dato',V.T. Sambanthan,NaN,male,indian,NaN,NaN,NaN,NaN
4,1,P092,Johor,Muar Selatan,Dato',Suleiman Bin Dato' Abdul Rahman,NaN,male,bumiputera,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841,11,P011,Kedah,Pendang,Dr.,Mohamed Hayati Bin Othman,NaN,male,bumiputera,NaN,NaN,NaN,NaN
1842,11,P066,Perak,Batu Gajah,Puan,Fong Po Kuan,NaN,female,NaN,NaN,NaN,NaN,NaN
1843,11,P043,Pulau Pinang,Bagan,Tuan,Lim Hock Seng,NaN,male,NaN,NaN,NaN,NaN,NaN
1844,11,P195,Sarawak,Bandar Kuching,Tuan,Chong Chieng Jen,NaN,male,NaN,NaN,NaN,NaN,NaN


In [706]:
final_df.to_csv("dr_1_11_merged.csv", index=False)

In [707]:
def parti_clean(parti_name):
    parti_split = parti_name.split(" ")
    if len(parti_split) > 1 and ">" not in parti_split:
        parti_name = f"{parti_split[0]} - {re.sub(r"[\(|\)]","",parti_split[1])}"
    return parti_name

In [708]:
table_combined = pd.read_csv("dr1_11_wiki_min.csv")

In [709]:
table_combined["parti"] = table_combined["parti"].str.replace("Bebas sejak 1963", "BEBAS")

In [710]:
table_combined["parti"] = table_combined["parti"].apply(parti_clean)

In [711]:
table_combined[["vol","kod_kawasan", "parti"]]

,vol,kod_kawasan,parti
0,1,P001,Perikatan - UMNO
1,1,P002,Perikatan - UMNO
2,1,P003,Perikatan - UMNO
3,1,P004,Perikatan - UMNO
4,1,P005,Perikatan - UMNO
...,...,...,...
1815,11,P215,BN - PRS
1816,11,P216,BN - SPDP
1817,11,P217,BN - SUPP
1818,11,P218,BN - SPDP


In [712]:
final_df[final_df[["vol","nama"]].duplicated()]

,vol,kod_kawasan,negeri,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti,jawatan_kabinet,jawatan_parlimen,gambar
75,1,P001,Perlis,Perlis Utara,Tuan,Othman Bin Abdullah,NaN,male,bumiputera,NaN,NaN,NaN,NaN
221,2,P001,Perlis,Perlis Utara,Tuan,Othman Bin Abdullah,NaN,male,bumiputera,NaN,NaN,NaN,NaN
1419,9,NaN,NaN,Gua Musang(mulai 4 September 1995),Tengku,Razaleigh Hamzah,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1501,10,P129,Johor,Mersing,Dr.,Abdul Latiff Bin Ahmad,NaN,male,bumiputera,NaN,NaN,NaN,NaN


In [673]:
pd.merge(final_df, table_combined[["vol","kod_kawasan", "parti"]].drop_duplicates(), "left", on=["vol","kod_kawasan"])

,index,vol,kod_kawasan,negeri,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti_x,jawatan_kabinet,jawatan_parlimen,gambar,parti_y
0,0,1,P007,Kedah,Kuala Kedah,Y.T.M. Tunku,Abdul Rahman Putra AlHaj,NaN,male,bumiputera,NaN,Perdana Menteri,NaN,NaN,Perikatan - UMNO
1,1,1,P062,Pahang,Pekan,Y.A.B. Tun,Abdul Razak Bin Dato' Hussain,NaN,male,bumiputera,NaN,Timbalan Perdana Menteri,NaN,NaN,Perikatan - UMNO
2,2,1,P102,Johor,Johor Timur,Dato' Dr.,Ismail Bin Dato' Abdul Rahman,NaN,male,bumiputera,NaN,NaN,NaN,NaN,Perikatan - UMNO
3,3,1,P046,Perak,Sungei Siput,Dato',V.T. Sambanthan,NaN,male,indian,NaN,NaN,NaN,NaN,Perikatan - MIC
4,4,1,P092,Johor,Muar Selatan,Dato',Suleiman Bin Dato' Abdul Rahman,NaN,male,bumiputera,NaN,NaN,NaN,NaN,Perikatan - UMNO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,1847,11,P011,Kedah,Pendang,Dr.,Mohamed Hayati Bin Othman,NaN,male,bumiputera,NaN,NaN,NaN,NaN,BA - PAS
1848,1848,11,P066,Perak,Batu Gajah,Puan,Fong Po Kuan,NaN,female,NaN,NaN,NaN,NaN,NaN,DAP
1849,1849,11,P043,Pulau Pinang,Bagan,Tuan,Lim Hock Seng,NaN,male,NaN,NaN,NaN,NaN,NaN,DAP
1850,1850,11,P195,Sarawak,Bandar Kuching,Tuan,Chong Chieng Jen,NaN,male,NaN,NaN,NaN,NaN,NaN,DAP


In [714]:
pd.merge(final_df, table_combined[["vol","kod_kawasan", "parti"]].drop_duplicates(), "left", on=["vol","kod_kawasan"]).to_csv("dr_1_11_merged.csv", index=False)

In [655]:
final_df.iloc[[  10,   31,   54,   73,   95,  154,  541,  553,  570,  625,  716,
        804,  883, 1228, 1323, 1347, 1374, 1406, 1418, 1422, 1519, 1621,
       1822]].sort_values(["vol","kod_kawasan"])

,vol,kod_kawasan,negeri,kawasan,gelaran,nama,tahun_lahir,jantina,kumpulan_etnik,parti,jawatan_kabinet,jawatan_parlimen,gambar
73,1,P027,Terengganu,Kuala Terengganu Selatan,Dato',Onn Bin Jaafar,NaN,male,bumiputera,NaN,NaN,NaN,NaN
54,1,P053,Perak,Kampar,Tuan,Leong Kee Nyean,NaN,male,chinese,NaN,NaN,NaN,NaN
95,1,P055,Perak,Telok Anson,Tuan,Woo Saik Hong,NaN,male,chinese,NaN,NaN,NaN,NaN
10,1,P075,Selangor,Kuala Langat,Tuan,Abdul Aziz bin Ishak,NaN,male,bumiputera,NaN,NaN,NaN,NaN
31,1,P080,Negeri Sembilan,Seremban Timur,Tuan,Chin See Yin,NaN,male,chinese,NaN,NaN,NaN,NaN
154,2,P071,Selangor,Bungsar,Tuan,C. V. Devan Nair,NaN,male,indian,NaN,NaN,NaN,NaN
541,4,P125,Sabah,Kimanis,NaN,Pengiran Tahir Bin Pengiran Patera,NaN,male,bumiputera,NaN,NaN,NaN,NaN
553,4,P127,Sabah,Keningau,Tuan,Stephen Robert Evans,NaN,male,NaN,NaN,NaN,NaN,NaN
570,5,P075,Selangor,Ulu Selangor,Dato',Michael Chen Wing Sum,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,5,P125,Sabah,Kimanis,Tuan,A. K. Aliuddin Bin Pengiran Haji Mohd. Tahir,NaN,male,bumiputera,NaN,NaN,NaN,NaN


In [657]:
table_combined.to_csv("dr_1_11_merged.csv", index=False)